# Super-Resolution Dataset Preparation
This notebook will:
1. Create the proper directory structure
2. Split the data into train (80%), validation (10%), and test (10%) sets
3. Extract frames from HR and LR videos

In [1]:
# Install required packages
!pip install opencv-python numpy tqdm

# Import libraries
import cv2
import os
import numpy as np
import random
from tqdm.notebook import tqdm
import time

In [2]:
# Configuration
# Input directories containing videos
hr_dir = '/teamspace/studios/this_studio/original_data/3_0x'
lr_dir = '/teamspace/studios/this_studio/original_data/1_0x'

# Output directory for dataset
output_dir = '/teamspace/studios/this_studio/sr_dataset/3_0x'

# Extract one frame every N frames (to reduce redundancy)
sample_rate = 5

# Random seed for reproducible splits
random_seed = 42

# Split ratios
train_ratio = 0.8
val_ratio = 0.1
test_ratio = 0.1  # Automatically calculated as 1 - train_ratio - val_ratio

In [3]:
# Setup Directory Structure
# Create main dataset directory
os.makedirs(output_dir, exist_ok=True)

# Create directories for train, val, test sets
train_dir = os.path.join(output_dir, 'train')
val_dir = os.path.join(output_dir, 'val')
test_dir = os.path.join(output_dir, 'test')

for split_dir in [train_dir, val_dir, test_dir]:
    hr_output_dir = os.path.join(split_dir, 'HR')
    lr_output_dir = os.path.join(split_dir, 'LR')
    os.makedirs(hr_output_dir, exist_ok=True)
    os.makedirs(lr_output_dir, exist_ok=True)
    
print(f"Created directory structure in {output_dir}")

Created directory structure in /teamspace/studios/this_studio/sr_dataset/3_0x


In [4]:
# Find and Match Video Files
# Find video files
hr_files = sorted([f for f in os.listdir(hr_dir) if f.endswith(('.mov', '.mp4', '.avi'))])
lr_files = sorted([f for f in os.listdir(lr_dir) if f.endswith(('.mov', '.mp4', '.avi'))])

print(f"Found {len(hr_files)} HR videos and {len(lr_files)} LR videos")

# Find matching pairs by filename
common_files = set(hr_files).intersection(set(lr_files))

if not common_files:
    print("No matching filenames found. Pairing by index...")
    # Pair videos by their position in the sorted list
    video_pairs = [(hr_files[i], lr_files[i]) for i in range(min(len(hr_files), len(lr_files)))]
else:
    print(f"Found {len(common_files)} matching filenames")
    video_pairs = [(f, f) for f in common_files]

print(f"Total video pairs to process: {len(video_pairs)}")

Found 95 HR videos and 95 LR videos
Found 95 matching filenames
Total video pairs to process: 95


In [5]:
# Split Videos into Train, Validation, and Test Sets
# Set random seed for reproducibility
random.seed(random_seed)

# Shuffle the video pairs
shuffled_pairs = video_pairs.copy()
random.shuffle(shuffled_pairs)

# Calculate split indices
total_videos = len(shuffled_pairs)
train_size = int(total_videos * train_ratio)
val_size = int(total_videos * val_ratio)
test_size = total_videos - train_size - val_size

# Split the videos
train_pairs = shuffled_pairs[:train_size]
val_pairs = shuffled_pairs[train_size:train_size + val_size]
test_pairs = shuffled_pairs[train_size + val_size:]

print(f"Split videos into:")
print(f"  Training set: {len(train_pairs)} videos ({len(train_pairs)/total_videos:.1%})")
print(f"  Validation set: {len(val_pairs)} videos ({len(val_pairs)/total_videos:.1%})")
print(f"  Test set: {len(test_pairs)} videos ({len(test_pairs)/total_videos:.1%})")

Split videos into:
  Training set: 76 videos (80.0%)
  Validation set: 9 videos (9.5%)
  Test set: 10 videos (10.5%)


In [6]:
# Extract Frames from Videos for Each Split - Function Definition
def extract_frames_for_split(video_pairs, split_name, hr_source_dir, lr_source_dir):
    """Extract frames from videos and save them in the appropriate split directory"""
    split_dir = os.path.join(output_dir, split_name)
    hr_output_dir = os.path.join(split_dir, 'HR')
    lr_output_dir = os.path.join(split_dir, 'LR')
    
    # For meta_info.txt
    meta_entries = []
    total_saved_frames = 0
    
    for pair_idx, (hr_filename, lr_filename) in enumerate(tqdm(video_pairs, desc=f"Processing {split_name} videos")):
        hr_path = os.path.join(hr_source_dir, hr_filename)
        lr_path = os.path.join(lr_source_dir, lr_filename)
        
        hr_cap = cv2.VideoCapture(hr_path)
        lr_cap = cv2.VideoCapture(lr_path)
        
        if not hr_cap.isOpened() or not lr_cap.isOpened():
            print(f"Could not open video pair: {hr_filename}, {lr_filename}")
            continue
            
        hr_frames = int(hr_cap.get(cv2.CAP_PROP_FRAME_COUNT))
        lr_frames = int(lr_cap.get(cv2.CAP_PROP_FRAME_COUNT))
        pair_frames = min(hr_frames, lr_frames)
        
        print(f"\nVideo {pair_idx+1}/{len(video_pairs)}: {hr_filename}")
        print(f"  {pair_frames} frames available")
        
        frame_count = 0
        saved_count = 0
        
        # Process frames
        with tqdm(total=pair_frames//sample_rate + 1, desc=f"Extracting frames") as pbar:
            while frame_count < pair_frames:
                hr_ret, hr_frame = hr_cap.read()
                lr_ret, lr_frame = lr_cap.read()
                
                if not hr_ret or not lr_ret:
                    break
                    
                # Save every Nth frame
                if frame_count % sample_rate == 0:
                    # Generate filenames with video info to avoid name collisions
                    video_name = os.path.splitext(hr_filename)[0]
                    frame_basename = f"{video_name}_frame{saved_count:06d}"
                    hr_output_filename = f"{frame_basename}.png"
                    lr_output_filename = f"{frame_basename}.png"
                    
                    hr_outpath = os.path.join(hr_output_dir, hr_output_filename)
                    lr_outpath = os.path.join(lr_output_dir, lr_output_filename)
                    
                    # Save images
                    cv2.imwrite(hr_outpath, hr_frame)
                    cv2.imwrite(lr_outpath, lr_frame)
                    
                    # Add to meta entries
                    meta_entries.append(f"{hr_output_filename} {lr_output_filename}")
                    
                    saved_count += 1
                    pbar.update(1)
                
                frame_count += 1
                
        hr_cap.release()
        lr_cap.release()
        
        print(f"  Extracted {saved_count} frames from video {pair_idx+1}")
        total_saved_frames += saved_count
    
    # Write meta info file
    meta_path = os.path.join(split_dir, 'meta_info.txt')
    with open(meta_path, 'w') as f:
        f.write('\n'.join(meta_entries))
    
    print(f"\nTotal frames extracted for {split_name}: {total_saved_frames}")
    print(f"Created meta_info.txt with {len(meta_entries)} entries")
    
    return total_saved_frames

In [7]:
# Extract frames for each split
print("Processing training set...")
train_frames = extract_frames_for_split(train_pairs, 'train', hr_dir, lr_dir)

print("\nProcessing validation set...")
val_frames = extract_frames_for_split(val_pairs, 'val', hr_dir, lr_dir)

print("\nProcessing test set...")
test_frames = extract_frames_for_split(test_pairs, 'test', hr_dir, lr_dir)

Processing training set...


Processing train videos:   0%|          | 0/76 [00:00<?, ?it/s]


Video 1/76: reading-3608-Original.mov
  90 frames available


Extracting frames:   0%|          | 0/19 [00:00<?, ?it/s]

  Extracted 18 frames from video 1

Video 2/76: jellyfish-110877-Original.mov
  90 frames available


...

  Extracted 18 frames from video 8

Video 9/10: 166393.mov
  90 frames available


Extracting frames:   0%|          | 0/19 [00:00<?, ?it/s]

  Extracted 18 frames from video 9

Video 10/10: nature-42420-Original.mov
  90 frames available


Extracting frames:   0%|          | 0/19 [00:00<?, ?it/s]

  Extracted 18 frames from video 10

Total frames extracted for test: 177
Created meta_info.txt with 177 entries


In [8]:
# Install BasicSR if not already installed
import sys
import os
import subprocess

print("Installing BasicSR...")
%pip install basicsr
print("BasicSR installed, importing modules...")

# After installation, we need to import modules
from basicsr.utils import scandir
from basicsr.utils.lmdb_util import make_lmdb_from_imgs
import os.path as osp
from tqdm.notebook import tqdm
import cv2
import numpy as np
import time


Installing BasicSR...


Note: you may need to restart the kernel to use updated packages.
BasicSR installed, importing modules...


ModuleNotFoundError: No module named 'torchvision.transforms.functional_tensor'

## Make LMDB using BASICSR

In [6]:
import os
import glob
from basicsr.utils.lmdb_util import make_lmdb_from_imgs

def create_lmdb(data_dir, lmdb_path):
    # Get list of image files
    img_path_list = sorted(glob.glob(os.path.join(data_dir, '*.png')))
    
    # Generate keys from filenames (without extensions)
    keys = [os.path.basename(img).split('.')[0] for img in img_path_list]
    
    print("Keys:", keys[:10])  # Print first 10 keys for debugging
    print("Image List:", img_path_list[:10])  # Print first 10 image paths
    # Create LMDB
    make_lmdb_from_imgs(
        data_path=data_dir,
        lmdb_path=lmdb_path,
        img_path_list=img_path_list,
        keys=keys,
        batch=5000,
        compress_level=1,
        multiprocessing_read=False,
        n_thread=1,
        map_size=None
    )
    print(f'LMDB dataset created at {lmdb_path}')
    


In [4]:

# Define paths
train_hr_folder = '/teamspace/studios/this_studio/sr_dataset/4_0x/train/HR'
train_lr_folder = '/teamspace/studios/this_studio/sr_dataset/4_0x/train/LR'
train_hr_lmdb = '/teamspace/studios/this_studio/sr_dataset/4_0x/train_lmdb/train_HR.lmdb'
train_lr_lmdb = '/teamspace/studios/this_studio/sr_dataset/4_0x/train_lmdb/train_LR.lmdb'

# Format of your images (png, jpg, etc.)
img_format = 'png'  # Change this if you're using a different format


In [9]:
create_lmdb(train_hr_folder, train_hr_lmdb)
create_lmdb(train_lr_folder, train_lr_lmdb)

Keys: ['000086_VTXHD_frame000000', '000086_VTXHD_frame000001', '000086_VTXHD_frame000002', '000086_VTXHD_frame000003', '000086_VTXHD_frame000004', '000086_VTXHD_frame000005', '000086_VTXHD_frame000006', '000086_VTXHD_frame000007', '000086_VTXHD_frame000008', '000086_VTXHD_frame000009']
Image List: ['/teamspace/studios/this_studio/sr_dataset/4_0x/train/HR/000086_VTXHD_frame000000.png', '/teamspace/studios/this_studio/sr_dataset/4_0x/train/HR/000086_VTXHD_frame000001.png', '/teamspace/studios/this_studio/sr_dataset/4_0x/train/HR/000086_VTXHD_frame000002.png', '/teamspace/studios/this_studio/sr_dataset/4_0x/train/HR/000086_VTXHD_frame000003.png', '/teamspace/studios/this_studio/sr_dataset/4_0x/train/HR/000086_VTXHD_frame000004.png', '/teamspace/studios/this_studio/sr_dataset/4_0x/train/HR/000086_VTXHD_frame000005.png', '/teamspace/studios/this_studio/sr_dataset/4_0x/train/HR/000086_VTXHD_frame000006.png', '/teamspace/studios/this_studio/sr_dataset/4_0x/train/HR/000086_VTXHD_frame000007.pn

Write nature-31377-1080p_frame000016:  72%|███████▏  | 986/1365 [49:24<26:05,  4.13s/chunk]         

: 